## Task 1

In [3]:
from google.cloud import bigquery
from google.oauth2 import service_account
import os

# Initialize BigQuery client with credentials
service_path = ""
service_file = 'wedge-project-405605-b2caa39a9f6d.json'  
gbq_proj_id = 'wedge-project-405605'  
private_key = service_path + service_file

credentials = service_account.Credentials.from_service_account_file(private_key)
client = bigquery.Client(credentials=credentials, project=gbq_proj_id)

# Specify your dataset in BigQuery
dataset_id = 'Transactions'  # Replace with your dataset ID

# Path to the folder containing the cleaned CSV files
folder_path = 'C:\\Users\\Dylan\\Desktop\\Wedge\\wedge-clean-files\\clean-files'  

# Iterate over each file in the folder
for filename in os.listdir(folder_path):
    if filename.endswith('.csv'):  # Check if the file is a CSV
        # Create a table_id from the filename (excluding the extension)
        table_id = filename[:-4]  # Removes the '.csv' from the filename

        file_path = os.path.join(folder_path, filename)
        table_ref = client.dataset(dataset_id).table(table_id)

        # Configure the load job
        job_config = bigquery.LoadJobConfig(
            source_format=bigquery.SourceFormat.CSV,
            skip_leading_rows=1,  # Assuming your CSV files have a header row
            autodetect=True,  # Auto-detect schema (can be set manually if needed)
            write_disposition=bigquery.WriteDisposition.WRITE_TRUNCATE,  # Overwrite existing data
        )

        # Start the load job
        with open(file_path, "rb") as source_file:
            job = client.load_table_from_file(source_file, table_ref, job_config=job_config)

        job.result()  # Wait for the job to complete

        print(f"Uploaded {filename} as table {table_id} to BigQuery")

print("All files have been uploaded.")